# BBH End-to-End Inference Analysis Plots

The key questions I'm trying to address with the plots below are:
- How *quickly* can I process a given volume of data at a given scale of compute?
- How much does processing that data at that scale *cost* me?
- How can I *optimize* my deployment to minimize the cost incurred at a given scale?

In describing how the data below was collected, it will be valuable to establish some more precise language.
- A **server** will refer to one instance of the Triton Inference Server deployed on a single GPU-equipped cluster **node**.
- A **client** will refer to one instance of a Triton Inference Server client deployed on a single CPU-based GCP **VM** instance. Each client has a unique server to which it sends requests, and each server is associated with the same number of client VMs.
- The **scale** of a deployment will refer to the total number of clients and servers, and the resources allocated to each, used in a deployment.
- A **frame** will refer to a [gravitational wave frame file](https://www.gw-openscience.org/read_in_c/), a format for encoding multiple **channels** of concurrent timeseries of a fixed length and their associated metadata. The data below was collected by processing 24 frames of length 4096 seconds.
- Samples in each frame are mapped onto a fixed time grid at read time according to the desired **sampling rate** (fixed at train time).
- In order to parallelize processing across multiple frames, each client is assigned a contiguous (in time) whole-numbered subset of the total. Note that the restriction to whole numbers means that _not all clients will be responsible for the same number of frames_, some will have one more to process than the others. This is a restriction I'm working on relaxing. Each client's subset of data will be referred to as a **stream**.
- DeepClean and BBH both expect as input fixed-length **kernels** of data, in this case one second long. Kernels are sampled from streams at a fixed interval termed the **kernel stride**, which is selected at inference time and parameterizes the inference deployment.
- In order to minimize network I/O, servers maintain the most recently inferred-upon kernel for each stream as a **state**, which the client updates by sending its stream to the server in kernel stride-length packets. Each of these **requests** triggers a new **inference** for that stream which returns a single BBH event probability estimate as its **response**.
- Each server host multiple models which it uses to produce its response.
    - `snapshotter` is the model responsible for updating the state and producing a new kernel. It does this for all the witness channels as well as for the strain channels at once.
    - `deepclean_h` and `deepclean_l` take a kernel of multiple witness channel measurements and produce an estimate of the noise at either the Hanford or Livingston detectors, respectively.
    - `postproc` subtracts the noise estimates from the strain channels.
    - `bbh` uses the cleaned strain channels to produce a single scalar estimate of the likelihood that the signature of a binary blackhole merger occurred in the kernel.
    - The end-to-end execution of these models is handled by an **ensemble model** called `gwe2e`. The execution of this model is not associated with any particular GPU and so is excluded from the data collected below.
- Each model is hosted on every GPU, and a single GPU can even perform concurrent execution of the same model up to a user-defined level of concurrency called the number of model **instances** per GPU.
- The number of `snapshotter` instances is equal to the number of streams (since each stream needs its own state to maintain and update).
- A model's measured **throughput** is the number of inferences it performed in a given interval on a single GPU, divided by the length of that interval. Its **aggregate throughput** is the sum of that model's throughput across _all_ GPUs. The unit for both of these quantities is **inferences per second**
- A model's **queue time** as measured on a given interval is the _average_ amount of time a request had to wait before inference was executed on it during that interval.

The data below was collected by requesting inference metrics from each server in a round-robin fashion during the inference run. The collected data is available in CSV format with the following columns
- `ip` - the IP address of the server. Used to index different servers.
- `step` - indexes the metrics request that a chunk of measurements come from. Also used to roughly align in time metric requests made in serial to different servers.
- `gpu_id` - the ID of the GPU executing the model's inference.
- `model` - the model that the metrics correspond to.
- `process` - subdivides each model's inference into multiple steps. Of particular importance are the `request` process, which measures the end-to-end execution of an inference, and the `queue` process, which measures how long requests spent waiting before execution.
- `time (us)` - the _total_ time spent executing the corresponding process over the previous interval in microseconds
- `interval` - the time since the last metrics request was made in seconds
- `count` - the number of times the corresponding process was executed during the previous interval
- `utilization` the current GPU utilization fraction at the time of the metrics request
- `average_time` - the average time spent executing the process during the previous interval in microseconds (`time (us)` / `count`)
- `throughput` - the rate at which inferences were executed during the interval, measured in inferences per second (`count` / `interval`)
- `time_since_start` - the time since the start of the run at which the metrics were sampled.

Each CSV is associated with a `RunConfig` object that encodes the unique parameters of the associated inference run, as well as with an 8 character hexadecimal string `RunConfig.id` that identifies the config and is used as the directory where it and its metrics are stored. The configs for which data is available are stored in `plot_utils.configs`, and the metrics collected for a given config can be loaded via `plot_utils.load_stats_for_config`.

In [1]:
import plot_utils

print(plot_utils.configs)

Loading BokehJS ...

[RunConfig(num_nodes=2, gpus_per_node=4, clients_per_node=4, instance_config=InstanceConfig(deepclean_h=6, deepclean_l=6, postproc=1, bbh=1), vcpus_per_gpu=16, kernel_stride=0.05, generation_rate=750.0), RunConfig(num_nodes=4, gpus_per_node=4, clients_per_node=4, instance_config=InstanceConfig(deepclean_h=6, deepclean_l=6, postproc=1, bbh=1), vcpus_per_gpu=16, kernel_stride=0.05, generation_rate=750.0), RunConfig(num_nodes=2, gpus_per_node=4, clients_per_node=3, instance_config=InstanceConfig(deepclean_h=6, deepclean_l=6, postproc=1, bbh=1), vcpus_per_gpu=16, kernel_stride=0.1, generation_rate=750.0), RunConfig(num_nodes=2, gpus_per_node=4, clients_per_node=4, instance_config=InstanceConfig(deepclean_h=5, deepclean_l=5, postproc=2, bbh=2), vcpus_per_gpu=16, kernel_stride=0.1, generation_rate=750.0), RunConfig(num_nodes=2, gpus_per_node=4, clients_per_node=5, instance_config=InstanceConfig(deepclean_h=5, deepclean_l=5, postproc=2, bbh=2), vcpus_per_gpu=16, kernel_stride=0.1, generation_

In [2]:
print(plot_utils.configs[0])

RunConfig 26b4090 {
	num_nodes: 2,
	gpus_per_node: 4,
	clients_per_node: 4,
	instance_config: {
		deepclean_h: 6,
		deepclean_l: 6,
		postproc: 1,
		bbh: 1
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.05,
	generation_rate: 750.0
}


In [3]:
plot_utils.load_stats_for_config(plot_utils.configs[0])

,ip,step,gpu_id,model,process,time (us),interval,count,utilization,average_time,throughput,time_since_start
0,104.154.79.197,5,3368220e-d117-c578-8943-a360b50fe80b,bbh,compute_infer,1.959570e+05,0.410116,74,0.05,2.648068e+03,180.436563,0.410116
1,104.154.79.197,6,3368220e-d117-c578-8943-a360b50fe80b,bbh,compute_infer,3.814140e+05,0.434412,221,0.05,1.725855e+03,508.733083,0.844529
2,104.154.79.197,7,3368220e-d117-c578-8943-a360b50fe80b,bbh,compute_infer,5.173240e+05,0.636556,300,0.05,1.724413e+03,471.286175,1.481085
3,104.154.79.197,8,3368220e-d117-c578-8943-a360b50fe80b,bbh,compute_infer,1.271672e+06,1.295400,773,0.38,1.645113e+03,596.726924,2.776485
4,104.154.79.197,9,3368220e-d117-c578-8943-a360b50fe80b,bbh,compute_infer,1.071710e+06,1.143180,614,0.38,1.745456e+03,537.098096,3.919665
...,...,...,...,...,...,...,...,...,...,...,...,...
175075,35.202.237.31,777,69a07076-0ac6-6c71-3f86-29c410939e61,snapshotter,request,1.195493e+10,1.195264,687,0.38,1.740165e+07,574.768485,378.592636
175076,35.202.237.31,778,69a07076-0ac6-6c71-3f86-29c410939e61,snapshotter,request,4.978526e+09,0.396517,286,0.38,1.740743e+07,721.280047,378.989153
175077,35.202.237.31,779,69a07076-0ac6-6c71-3f86-29c410939e61,snapshotter,request,4.353299e+09,0.632773,254,0.35,1.713897e+07,401.407670,379.621927
175078,35.202.237.31,780,69a07076-0ac6-6c71-3f86-29c410939e61,snapshotter,request,4.915411e+09,0.449315,305,0.35,1.611610e+07,678.811332,380.071241


You can retrieve all the configs matching some desired set of criteria by using `plot_utils.get_configs`, e.g.

In [4]:
for config in plot_utils.get_configs(num_nodes=4):
    print(config)

RunConfig 3f14092 {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 4,
	instance_config: {
		deepclean_h: 6,
		deepclean_l: 6,
		postproc: 1,
		bbh: 1
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.05,
	generation_rate: 750.0
}
RunConfig c58d405d {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 3,
	instance_config: {
		deepclean_h: 6,
		deepclean_l: 6,
		postproc: 1,
		bbh: 1
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.1,
	generation_rate: 750.0
}
RunConfig c5ec405e {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 4,
	instance_config: {
		deepclean_h: 5,
		deepclean_l: 5,
		postproc: 2,
		bbh: 2
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.1,
	generation_rate: 750.0
}
RunConfig c626405e {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 4,
	instance_config: {
		deepclean_h: 6,
		deepclean_l: 6,
		postproc: 1,
		bbh: 1
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.1,
	generation_rate: 750.0
}
RunConfig c6bf405f {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 5,
	instance_c

In [5]:
for config in plot_utils.get_configs(num_nodes=4, clients_per_node=5):
    print(config)

RunConfig c6bf405f {
	num_nodes: 4,
	gpus_per_node: 4,
	clients_per_node: 5,
	instance_config: {
		deepclean_h: 6,
		deepclean_l: 6,
		postproc: 1,
		bbh: 1
	},
	vcpus_per_gpu: 16,
	kernel_stride: 0.1,
	generation_rate: 750.0
}


Let's start with the first question: how long does it take to process a given amount of data given a certain level of scale? To borrow Erik's terminology, we can look at this in terms of a multipole expansion around the parameter space.

At the simplest level, we can search over all the different runs done at a given level of scale, find the configuration that ran in the shortest time, then compare this across levels of scale. We'll also compare across kernel strides, since that will obviously dictate the total number of inferences that we need to do. (Though at time of writing, I've only generated data for kernel strides of 100 ms, so this plot won't be quite as interesting.)

I'll note up front that much of this code could be made cleaner and more modular, and I hope to do that sometime this week, but my focus was just on getting these plots together, so I apologize if the code is difficult to make sense of.

In [6]:
import numpy as np
from collections import defaultdict
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.io import show

times_to_run = defaultdict(lambda : np.inf)
color_map = {}
color_iter = iter(plot_utils.palette)
for config in plot_utils.configs:
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.model == "bbh") & (df.process == "request")]
    time_to_run = df["time_since_start"].max()

    index = (config.num_nodes, config.total_clients, config.kernel_stride)
    times_to_run[index] = min(times_to_run[index], time_to_run)
    if config.kernel_stride not in color_map:
        color_map[config.kernel_stride] = next(color_iter)

x = sorted(times_to_run.keys())
counts = [times_to_run[key] for key in x]
colors = [color_map[key[2]] for key in x]

def _make_range(nodes, clients, stride):
    return (f"{nodes} nodes", f"{clients} streams", f"{stride} s")
x = [_make_range(*key) for key in x]

p = plot_utils.make_figure(
    title="Time to Run vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Time to run (s)",
    x_range=FactorRange(*x)
)

source = ColumnDataSource({"x": x, "counts": counts, "colors": colors})
p.vbar(
    x="x",
    top="counts",
    width=0.9,
    fill_color="colors",
    line_color="colors",
    fill_alpha=0.8,
    source=source
)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

Of course, this might not be the world's most helpful metric if you have a different volume of data that you need to process. So it would perhaps be more insightful to scale by the volume of data as measured in seconds, making the unit along the y-axis *seconds per second of data*.

In [7]:
from bokeh.models import PrintfTickFormatter

seconds_per_seconds = defaultdict(lambda : np.inf)
total_time_of_data = 4096 * 24
for config in plot_utils.configs:
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.model == "bbh") & (df.process == "request")]
    time_to_run = df["time_since_start"].max()

    index = (config.num_nodes, config.total_clients, config.kernel_stride)
    seconds_per_second = time_to_run / total_time_of_data
    seconds_per_seconds[index] = min(seconds_per_seconds[index], seconds_per_second)

x = sorted(seconds_per_seconds.keys())
counts = [seconds_per_seconds[key] for key in x]
colors = [color_map[key[2]] for key in x]

def _make_range(nodes, clients, stride):
    return (f"{nodes} nodes", f"{clients} streams", f"{stride} s")
x = [_make_range(*key) for key in x]

p = plot_utils.make_figure(
    title="Time to Run Per Second of Data vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Time to run per second of data (s / s')",
    x_range=FactorRange(*x)
)
p.yaxis[0].formatter = PrintfTickFormatter(format="%4.0e")

source = ColumnDataSource({"x": x, "counts": counts, "colors": colors})
p.vbar(
    x="x",
    top="counts",
    width=0.9,
    fill_color="colors",
    line_color="black",
    fill_alpha=0.8,
    source=source
)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

If these point estimates are just the monopole expansion, we can get to the next order by sampling more densely from the distributions which generated these points and plotting the resulting empirical distributions as violin plots. Unfortunately, that would be time and cost intensive to get anywhere near the level of density which would make for a useful plot.

However, we can imagine that these total time (and let's call it _time density_) measurements are really the result of summing a bunch of draws from a _throughput_ distribution, a distribution that we _do_ already have lots of samples from.

A quick note that Bokeh won't support grouping factors here, so it won't be quite as readable, but still manages to the point across I think.

In [8]:
def make_violin_patch(config, percentile=5, bins=25):
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.process == "request") & (df.model == "bbh")]
    inferences_per_second = df.groupby("step")["throughput"].agg("sum")
    seconds_per_second = 1 / (inferences_per_second * config.kernel_stride)

    # TODO: use some sort of KDE instead?
    min_, max_ = np.percentile(seconds_per_second, [percentile, 100-percentile])
    diff = (max_ - min_) / (bins + 1)
    bins = np.linspace(min_ - diff, max_ + diff, bins + 3)
    counts, _ = np.histogram(seconds_per_second, bins)
    density = counts / (counts.max() * 1.05)
    density[0] = density[-1] = 0
    bins = (bins[:-1] + bins[1:]) / 2

    xs = list(density / 2) + list(-density[::-1] / 2)
    ys = list(bins) + list(bins[::-1])
    return xs, ys

seconds_per_seconds = defaultdict(lambda : np.inf)
best_configs = {}
for config in plot_utils.configs:
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.model == "bbh") & (df.process == "request")]
    time_to_run = df["time_since_start"].max()

    index = (config.num_nodes, config.total_clients, config.kernel_stride)
    seconds_per_second = time_to_run / total_time_of_data
    if seconds_per_second < seconds_per_seconds[index]:
        seconds_per_seconds[index] = seconds_per_second
        best_configs[index] = config

x = sorted(seconds_per_seconds.keys())
configs = [best_configs[key] for key in x]
colors = [color_map[key[2]] for key in x]

def _make_range(nodes, streams, stride):
    return f"{nodes} nodes\n{streams} streams\n{stride} stride"
x = [_make_range(*key) for key in x]

p = plot_utils.make_figure(
    title="Time to Run Per Second of Data vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Time to run per second of data (s / s')",
    x_range=FactorRange(*x)
)
p.yaxis[0].formatter = PrintfTickFormatter(format="%4.0e")

source = ColumnDataSource()
for i, (key, color, config) in enumerate(zip(x, colors, configs)):
    x_col, y_col = f"x_{i}", f"y_{i}"
    xs, ys = make_violin_patch(config, percentile=5)
    source.add(ys, y_col)

    xs = list(zip([key]*len(xs), xs))
    source.add(xs, x_col)

    p.patch(
        x_col,
        y_col,
        color=color,
        alpha=0.6,
        line_color="black",
        source=source
    )

show(p)

Getting from the first question of time to the second one of cost is reasonably straightforward: just input the cost per resource per unit time then multiply by the time density.

In [9]:
cost_per_n1_cpu_per_hour = 0.04749975
cpus_per_client = 8
cost_per_gpu_per_hour = 0.35

def map_to_cost(seconds_per_second, config):
    cost_per_gpu = cost_per_gpu_per_hour + cost_per_n1_cpu_per_hour * config.vcpus_per_gpu
    server_cost = config.gpus_per_node * cost_per_gpu
    return seconds_per_second * server_cost / 3600
    

cost_per_seconds = defaultdict(lambda : np.inf)
for config in plot_utils.configs:
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.model == "bbh") & (df.process == "request")]
    time_to_run = df["time_since_start"].max()

    index = (config.num_nodes, config.total_clients, config.kernel_stride)
    seconds_per_second = time_to_run / total_time_of_data
    cost_per_second = map_to_cost(seconds_per_second, config)

    # this time I'm just going to collect the winning
    # configs up front that way I don't have to loop
    # through again later
    if cost_per_second < cost_per_seconds[index]:
        cost_per_seconds[index] = cost_per_second
        best_configs[index] = config

x = sorted(cost_per_seconds.keys())
costs = [cost_per_seconds[key] for key in x]
colors = [color_map[key[2]] for key in x]

def _make_range(nodes, clients, stride):
    return (f"{nodes} nodes", f"{clients} streams", f"{stride} s")

factors = [_make_range(*key) for key in x]
p = plot_utils.make_figure(
    title="Cost Per Second of Data vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Cost per second of data ($US / s')",
    x_range=FactorRange(*factors)
)
p.yaxis[0].formatter = PrintfTickFormatter(format="%4.0e")

source = ColumnDataSource({"x": factors, "costs": costs, "colors": colors})
p.vbar(
    x="x",
    top="costs",
    width=0.9,
    fill_color="colors",
    line_color="black",
    fill_alpha=0.8,
    source=source
)

p.y_range.start = 0
p.x_range.range_padding = 0.1
p.xaxis.major_label_orientation = 1
p.xgrid.grid_line_color = None

show(p)

In [10]:
def make_violin_patch(config, percentile=5, bins=25):
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.process == "request") & (df.model == "bbh")]
    inferences_per_second = df.groupby("step")["throughput"].agg("sum")
    seconds_per_second = 1 / (inferences_per_second * config.kernel_stride)
    cost_per_second = map_to_cost(seconds_per_second, config)

    # TODO: use some sort of KDE instead?
    min_, max_ = np.percentile(cost_per_second, [percentile, 100-percentile])
    diff = (max_ - min_) / (bins + 1)
    bins = np.linspace(min_ - diff, max_ + diff, bins + 3)
    counts, _ = np.histogram(cost_per_second, bins)
    density = counts / (counts.max() * 1.05)
    density[0] = density[-1] = 0
    bins = (bins[:-1] + bins[1:]) / 2

    xs = list(density / 2) + list(-density[::-1] / 2)
    ys = list(bins) + list(bins[::-1])
    return xs, ys

configs = [best_configs[key] for key in x]

def _make_range(nodes, streams, stride):
    return f"{nodes} nodes\n{streams} streams\n{stride} stride"

p = plot_utils.make_figure(
    title="Time to Run Per Second of Data vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Cost per second of data ($US / s')",
    x_range=FactorRange(*[_make_range(*key) for key in x])
)
p.yaxis[0].formatter = PrintfTickFormatter(format="%4.0e")

source = ColumnDataSource()
for i, (key, color, config) in enumerate(zip(x, colors, configs)):
    x_col, y_col = f"x_{i}", f"y_{i}"
    xs, ys = make_violin_patch(config, percentile=5)
    source.add(ys, y_col)

    key = _make_range(*key)
    xs = list(zip([key]*len(xs), xs))
    source.add(xs, x_col)

    p.patch(
        x_col,
        y_col,
        color=color,
        alpha=0.6,
        line_color="black",
        source=source
    )

show(p)

As we might have expected, this plot looks an awful lot like the violin plots for the time density, since each individual violin is just scaled by a constant factor. This might invite the idea of just putting a second y-axis and keeping everything the same, but the issue is that each violin is scaled by a different factor (and as our configurations got more exotic or varied, you can imagine how this might change even more). However, this does leave the possibility of plotting asymmetric violins, with one side plotting the cost distribution and the other side plotting the time distribution. To do this, I'll stop using colors to denote kernel strides and instead use them to color code the sides of the violin to the axes.

In [11]:
from bokeh.models import LinearAxis, Range1d

def make_violin_patch(config, percentile=5, bins=25):
    df = plot_utils.load_stats_for_config(config)
    df = df[(df.process == "request") & (df.model == "bbh")]
    inferences_per_second = df.groupby("step")["throughput"].agg("sum")
    seconds_per_second = 1 / (inferences_per_second * config.kernel_stride)
    cost_per_second = map_to_cost(seconds_per_second, config)

    outputs = []
    num_bins = bins
    for i, metric in enumerate([seconds_per_second, cost_per_second]):
        min_, max_ = np.percentile(metric, [percentile, 100-percentile])
        diff = (max_ - min_) / (num_bins + 1)
        bins = np.linspace(min_ - diff, max_ + diff, num_bins + 3)
        counts, _ = np.histogram(metric, bins)
    
        sign = (-1)**(i+1)
        density = sign * counts / (2 * counts.max() * 1.05)
        density[0] = density[-1] = 0
        bins = (bins[:-1] + bins[1:]) / 2

        outputs.append([density, bins])
    return outputs

old_range = p.y_range
p = plot_utils.make_figure(
    title="Time to Run Per Second of Data vs. Scale",
    x_axis_label="Configuration",
    y_axis_label="Time to run per second of data (s / s')",
    x_range=FactorRange(*[_make_range(*key) for key in x]),
)
p.yaxis[0].formatter = PrintfTickFormatter(format="%4.0e")
p.yaxis[0].axis_label_text_color = plot_utils.palette[0]

max_cost = max([max(v) for k, v in source.data.items() if k.startswith("y")])
p.extra_y_ranges = {"cost": Range1d(start=0, end=max_cost)}
p.add_layout(
    LinearAxis(
        y_range_name="cost",
        axis_label="Cost per second of data ($US / s')",
        axis_label_text_color=plot_utils.palette[1],
        formatter=p.yaxis[0].formatter,
    ),
    "right"
)

source = ColumnDataSource()
for i, (key, color, config) in enumerate(zip(x, colors, configs)):
    x_time_col, y_time_col = f"x_time_{i}", f"y_time_{i}"
    x_cost_col, y_cost_col = f"x_cost_{i}", f"y_cost_{i}"

    (x_time, y_time), (x_cost, y_cost) = make_violin_patch(config, percentile=5)
    source.add(y_time, y_time_col)
    source.add(y_cost, y_cost_col)

    key = _make_range(*key)
    x_time = list(zip([key]*len(x_time), x_time))
    x_cost = list(zip([key]*len(x_cost), x_cost))
    source.add(x_time, x_time_col)
    source.add(x_cost, x_cost_col)

    p.patch(
        x_time_col,
        y_time_col,
        color=plot_utils.palette[0],
        alpha=0.6,
        line_color="black",
        source=source
    )
    p.patch(
        x_cost_col,
        y_cost_col,
        color=plot_utils.palette[1],
        alpha=0.6,
        line_color="black",
        source=source,
        y_range_name="cost"
    )

p.y_range.start = 0
show(p)

The distributions are still a bit dodgy due to the whole-number enforcement on the frames per client, but I think this graph once appropriately filled out should capture all the 2nd order information of the first two questions.

In the next couple of days I will start putting together what graphs I think help answer the optimization question.

In [12]:
configs = plot_utils.get_configs(num_nodes=4, clients_per_node=4, kernel_stride=0.1)
configs

[RunConfig(num_nodes=4, gpus_per_node=4, clients_per_node=4, instance_config=InstanceConfig(deepclean_h=5, deepclean_l=5, postproc=2, bbh=2), vcpus_per_gpu=16, kernel_stride=0.1, generation_rate=750.0),
 RunConfig(num_nodes=4, gpus_per_node=4, clients_per_node=4, instance_config=InstanceConfig(deepclean_h=6, deepclean_l=6, postproc=1, bbh=1), vcpus_per_gpu=16, kernel_stride=0.1, generation_rate=750.0)]

In [16]:
config = plot_utils.get_configs(num_nodes=2, kernel_stride=0.05)[0]
plot_utils.plot_throughput_vs_time(config)

In [14]:
df = plot_utils.load_stats_for_config(config)
for ip, d in df.groupby("ip"):
    print(ip)
    plot_utils.plot_throughput_vs_time(d)

34.127.6.60


34.145.10.176


34.145.4.1


34.145.74.73


In [15]:
import glob
import os
import numpy as np
for f in glob.glob(os.path.join("data", config.id, "*.npy")):
    x = np.load(f)
    print(f, x.shape)

data\c626405e\1185402880.0-6145.0.npy (61449,)
data\c626405e\1185409024.0-6145.0.npy (61449,)
data\c626405e\1185415168.0-6145.0.npy (61449,)
data\c626405e\1185421312.0-6145.0.npy (61449,)
data\c626405e\1185427456.0-6145.0.npy (61449,)
data\c626405e\1185433600.0-6145.0.npy (61448,)
data\c626405e\1185439744.0-6145.0.npy (61449,)
data\c626405e\1185445888.0-6145.0.npy (61444,)
data\c626405e\1185452032.0-6145.0.npy (61449,)
data\c626405e\1185458176.0-6145.0.npy (61427,)
data\c626405e\1185464320.0-6145.0.npy (61449,)
data\c626405e\1185470464.0-6145.0.npy (61449,)
data\c626405e\1185476608.0-6145.0.npy (61435,)
data\c626405e\1185482752.0-6145.0.npy (61449,)
data\c626405e\1185488896.0-6145.0.npy (61431,)
data\c626405e\1185495040.0-6145.0.npy (61439,)
data\c626405e\H-H1_HOFT_C02-1185402880_1185406976-4096.npy (81919,)
data\c626405e\H-H1_HOFT_C02-1185411072_1185415168-4096.npy (81918,)
data\c626405e\H-H1_HOFT_C02-1185419264_1185423360-4096.npy (81918,)
data\c626405e\H-H1_HOFT_C02-1185427456_11854